In [89]:
import tensorflow.keras as keras
import numpy as np
from sklearn.utils import resample
import pandas as pd
import pickle
import os
import csv
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from keras.regularizers import l2
MODEL_PATH = './model/'
DATA_PATH = './data/'

In [118]:
pur_data = np.load(DATA_PATH+'purchase100.npz')
features = pur_data['features']
labels = pur_data['labels']
data = pd.DataFrame(features[:,:])
labels = np.argmax(labels, axis=1)
data['600'] = labels

In [119]:
target_dataset = pd.DataFrame(data.iloc[1:20001,].values)
shadow_dataset = pd.DataFrame(data.iloc[15001:50001,].values)
attack_test_nonmembers = pd.DataFrame(data.iloc[75001:85001,].values)
attack_test_members = pd.DataFrame(data.iloc[5001:15001,].values)

In [ ]:
data = pd.read_csv('data/adult.data', na_values=["?"])
target_dataset = pd.DataFrame(data.iloc[1:10001,].values)
shadow_dataset = pd.DataFrame(data.iloc[15001:30001,].values)
attack_test_nonmembers = pd.DataFrame(data.iloc[11001:14001,].values)
attack_test_members = pd.DataFrame(data.iloc[2001:5001,].values)

In [ ]:
target_dataset = pd.read_csv('data/adult_sds.csv', na_values=["?"])
shadow_dataset = pd.read_csv('data/adultODS10K_to_25K.csv', na_values=["?"])
attack_test_nonmembers = pd.read_csv('data/adultODS25K_to_28K.csv', na_values=["?"])
attack_test_members = pd.read_csv('data/adultODS1_to_3K.csv', na_values=["?"])

In [92]:
def read_data(data_name):
    with np.load(DATA_PATH + data_name) as f:
        train_x, train_y, test_x, test_y = [f['arr_%d' % i] for i in range(len(f.files))]
    return train_x, train_y, test_x, test_y

In [93]:
def transform_data(dataset):
    x = dataset.iloc[:,0:dataset.shape[1]-1] # seperate the feature column from class label
    y = dataset.iloc[:,-1] # label column

    dim=x.shape[1] # number of feature columns

    #num of classes
    num_classes=2

    for j in range(0,dim):
        if x.iloc[:,j].dtypes == object:   # transform categorical variables
            x.iloc[:,j] = x.iloc[:,j].astype('category') # change datatype to categorical
            x.iloc[:,j] = x.iloc[:,j].cat.codes # change from category name to category number
        else:  #transform numrical variables to standard scaler form 
            sc = StandardScaler()  
            val=np.array(x.iloc[:,j]).reshape(-1,1)
            std_data = sc.fit_transform(val)
            std_data = pd.DataFrame(std_data)
            x.iloc[:,j]=std_data

    y = y.astype('category') # change label to categorical
    y = y.cat.codes # change from category name to number

    x=np.array(x)
    y=np.array(y)
    
    
    y=to_categorical(y)
    
    return x, y, dim

In [94]:
def transform_puchase_data(dataset): 
    df_tot = dataset
    df_tot.dropna(inplace=True)

    trainX = df_tot.iloc[:,0:dataset.shape[1]-1]
    trainY = df_tot.iloc[:,-1]

    dim=trainX.shape[1]


    #num of classes
    num_classes=100

    trainX=np.array(trainX)
    trainY=np.array(trainY)
    
    trainY = to_categorical(trainY)


    return trainX, trainY, dim

In [95]:
def load_target_data(dataset, test_ratio):
    x, y, dim = transform_puchase_data(dataset)
    
    #trainX,testX, trainY, testY = train_test_split(x, y, test_size=test_ratio, random_state=0, stratify=y)
    trainX = x[0:14000,]
    testX = x[14000:,]
    trainY = y[0:14000,]
    testY = y[14000:,]
    return (trainX, trainY), (testX, testY), dim

In [108]:
def build_purchase_dnn(n_class,dim):
    model = Sequential()
    
    model.add(Dense(600, input_dim=dim))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    #model.add(Dense(1024), kernel_regularizer=l2(0.001))
    #model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(512, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))

    model.add(Dense(256, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(128, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    
    #opt = SGD(lr=0.01, momentum=0.9)
    #model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    act_layer=6
    
    return model, act_layer

In [97]:
def build_simple_mlp(n_class,pix,d):

    model = Sequential()
    model.add(Dense(256, input_dim=pix))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(256, kernel_regularizer=l2(0.01)))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    
    #model.add(Dense(248))
    #model.add(Activation("relu"))
    #model.add(Dropout(0.01))

    model.add(Dense(64))
    model.add(Activation("relu"))
    model.add(Dropout(0.01))
    
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=3
    
    return model, act_layer

In [98]:
def load_shadow_data(dataset, n_shadow, shadow_size, test_ratio):
    x, y, _ = transform_puchase_data(dataset)
    
    shadow_indices = np.arange(len(dataset))
    
   
    for i in range(n_shadow):
        shadow_i_indices = np.random.choice(shadow_indices, shadow_size, replace=False)
        shadow_i_x, shadow_i_y = x[shadow_i_indices], y[shadow_i_indices]
        trainX,testX, trainY, testY = train_test_split(shadow_i_x, shadow_i_y, test_size=test_ratio)
        print('shadow_i_trainX = ', len(trainX), 'shadow_i_trainY = ', len(trainY), 'shadow_i_testX = ', len(testX), 'shadow_i_testY = ', len(testY))
        
        np.savez(DATA_PATH + 'shadow_purchase{}_data.npz'.format(i), trainX, trainY, testX, testY)

In [99]:
def train_shadow_models(n_shadow, n_class, dim, channel):
    full_sm_train_pred=[]
    full_sm_train_class=[]
    
    full_sm_test_pred=[]
    full_sm_test_class=[]
    
    full_clz_train=[]
    full_clz_test=[]
    
    members=[]
    nonmembers=[]

    for j in range(n_shadow):
        
        print("Shadow Model ", j)
        
        print('Training shadow model {}'.format(j))
        data = read_data('shadow_purchase{}_data.npz'.format(j))
        x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test = data
        #print('x_shadow trian\n', x_shadow_train,'\n y_shadow trian\n', y_shadow_train, '\n x_shadow test\n', x_shadow_test, '\n y_shadow test\n', y_shadow_test)

        model, act_layer = build_purchase_dnn(n_class,dim)
            
            
        # fit model
        history = model.fit(x_shadow_train, y_shadow_train, epochs=EPS, batch_size=32, validation_data=(x_shadow_test, y_shadow_test), verbose=0)
    
        # evaluate model
        _, train_acc = model.evaluate(x_shadow_train, y_shadow_train, verbose=0)
        _, test_acc = model.evaluate(x_shadow_test, y_shadow_test, verbose=0)
        print("Shadow Train acc : ", (train_acc * 100.0),"Shadow Test acc : ", (test_acc * 100.0))

    
        #train SM
        sm_train_pred=model.predict(x_shadow_train, batch_size=32)
        sm_train_class=np.argmax(y_shadow_train,axis=1)
    
    
        #test SM
        sm_test_pred=model.predict(x_shadow_test, batch_size=32)
        sm_test_class=np.argmax(y_shadow_test,axis=1)
        
     
        full_sm_train_pred.append(sm_train_pred)        
        full_sm_train_class.append(sm_train_class)
        members.append(np.ones(len(sm_train_pred)))
        
        full_sm_test_pred.append(sm_test_pred)        
        full_sm_test_class.append(sm_test_class) 
        nonmembers.append(np.zeros(len(sm_test_pred)))


    full_sm_train_pred = np.vstack(full_sm_train_pred)
    full_sm_train_class = [item for sublist in full_sm_train_class for item in sublist]
    members = [item for sublist in members for item in sublist]
    
    full_sm_test_pred = np.vstack(full_sm_test_pred)
    full_sm_test_class = [item for sublist in full_sm_test_class for item in sublist]
    nonmembers = [item for sublist in nonmembers for item in sublist]
    
    shadow_train_performance=(full_sm_train_pred, np.array(full_sm_train_class))
    shadow_test_performance=(full_sm_test_pred, np.array(full_sm_test_class))


    ###atack data preparation
    attack_x = (full_sm_train_pred,full_sm_test_pred)
    #attack_x = np.vstack(attack_x)
    
    attack_y = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
    #attack_y = np.concatenate(attack_y)
    #attack_y = attack_y.astype('int32')
    
    
    classes = (np.array(full_sm_train_class),np.array(full_sm_test_class))
    #classes = np.array([item for sublist in classes for item in sublist])


    attack_dataset = (attack_x,attack_y,classes)

            
    return  shadow_train_performance, shadow_test_performance, attack_dataset, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, model

In [100]:
def define_attack_model(n_class):
    model = Sequential()
    
    model.add(Dense(1))
    model.add(Activation("relu"))
    
    model.add(Dense(1))
    model.add(Activation("relu"))

    model.add(Dense(n_class, activation='softmax'))

    # compile model
    opt = SGD(learning_rate=0.0001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [101]:
def attack_mlp(pix,d):

    model = Sequential()
    model.add(Dense(64, input_dim=pix))
    model.add(Activation("relu"))
    #model.add(Dropout(0.1))

#     model.add(Dense(32))
#     model.add(Activation("tanh"))
#     model.add(Dropout(0.01))
    
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=1
    
    return model, act_layer

In [102]:
def prep_attack_data(n_attack_data):

    attack_mem = pd.DataFrame(n_attack_data[0][0])
    attack_nmem = pd.DataFrame(n_attack_data[0][1])
    
    attack_mem_status = pd.DataFrame(n_attack_data[1][0])
    attack_mem_status.columns = ["membership"]
    
    attack_nmem_status = pd.DataFrame(n_attack_data[1][1])
    attack_nmem_status.columns = ["membership"]
    
    real_class_mem = pd.DataFrame(n_attack_data[2][0])
    real_class_mem.columns = ["y"]
    
    real_class_nmem = pd.DataFrame(n_attack_data[2][1])
    real_class_nmem.columns = ["y"]

    memdf = pd.concat([attack_mem,attack_nmem],axis=0)
    memdf = memdf.reset_index(drop=True)

    memstatus =  pd.concat([attack_mem_status,attack_nmem_status],axis=0)
    memstatus = memstatus.reset_index(drop=True)

    realclass = pd.concat([real_class_mem,real_class_nmem],axis=0)
    realclass = realclass.reset_index(drop=True)

    attack_df = pd.concat([memdf,realclass,memstatus],axis=1)
    
    return attack_df

In [133]:
def prep_validation_data(attack_test_data):

    attack_mem = pd.DataFrame(attack_test_data[0][0])
    attack_nmem = pd.DataFrame(attack_test_data[0][1])
    
    attack_mem_status = pd.DataFrame(attack_test_data[1][0])
    attack_mem_status.columns = ["membership"]
    
    attack_nmem_status = pd.DataFrame(attack_test_data[1][1])
    attack_nmem_status.columns = ["membership"]
    
    real_class_mem = pd.DataFrame(attack_test_data[2][0])
    real_class_mem.columns = ["y"]
    
    real_class_nmem = pd.DataFrame(attack_test_data[2][1])
    real_class_nmem.columns = ["y"]
    
    mem_df = pd.concat([attack_mem,real_class_mem],axis=1)
    nmem_df = pd.concat([attack_nmem,real_class_nmem],axis=1)

#     memdf = pd.concat([attack_mem,attack_nmem],axis=0)
#     memdf = memdf.reset_index(drop=True)

#     memstatus =  pd.concat([attack_mem_status,attack_nmem_status],axis=0)
#     memstatus = memstatus.reset_index(drop=True)

#     realclass = pd.concat([real_class_mem,real_class_nmem],axis=0)
#     realclass = realclass.reset_index(drop=True)

#     attack_df = pd.concat([memdf,realclass,memstatus],axis=1)
    
    return mem_df, nmem_df

In [103]:
def load_attack_test_data(members, nonmembers):
    memberX, memberY, _ = transform_puchase_data(members)
    
    nonmemberX, nonmemberY, _ = transform_puchase_data(nonmembers)
    
    return memberX, memberY, nonmemberX, nonmemberY

In [104]:
def prety_print_result(mem, pred):
    tn, fp, fn, tp = confusion_matrix(mem, pred).ravel()
    print('TP: %d     FP: %d     FN: %d     TN: %d' % (tp, fp, fn, tn))
    if tp == fp == 0:
        print('PPV: 0\nAdvantage: 0')
    else:
        print('PPV: %.4f\nAdvantage: %.4f' % (tp / (tp + fp), tp / (tp + fn) - fp / (tn + fp)))

    return tp, fp, fn, tn, (tp / (tp + fp)), (tp / (tp + fn) - fp / (tn + fp)), ((tp+tn)/(tp+tn+fp+fn)),  (tp / (tp + fn))

In [139]:
def train_attack_model(attack_data, check_membership, n_hidden=50, learning_rate=0.01, batch_size=200, epochs=50, model='nn', l2_ratio=1e-7):

    x, y,  classes = attack_data

    train_x = x[0]
    train_y = y[0]
    test_x = x[1]
    test_y = y[1]
    train_classes = classes[0]
    test_classes = classes[1]
    
    
    checkmem_prediction_vals, checkmem_membership_status, checkmem_class_status = check_membership
    
    checkmem_prediction_vals=np.vstack(checkmem_prediction_vals)
    checkmem_membership_status=np.array([item for sublist in checkmem_membership_status for item in sublist])
    checkmem_class_status=np.array([item for sublist in checkmem_class_status for item in sublist])
    
    train_indices = np.arange(len(train_x))
    test_indices = np.arange(len(test_x))
    unique_classes = np.unique(train_classes)


    predicted_membership, target_membership = [], []
    for c in unique_classes:
        print("Class : ", c)
        c_train_indices = train_indices[train_classes == c]
        c_train_x, c_train_y = train_x[c_train_indices], train_y[c_train_indices]
        c_test_indices = test_indices[test_classes == c]
        c_test_x, c_test_y = test_x[c_test_indices], test_y[c_test_indices]
        c_dataset = (c_train_x, c_train_y, c_test_x, c_test_y)        
        
        full_cx_data=(c_train_x,c_test_x)
        full_cx_data = np.vstack(full_cx_data)

        full_cy_data=(c_train_y,c_test_y)
        full_cy_data = np.array([item for sublist in full_cy_data for item in sublist])
        
#         over_sampler = SMOTE(k_neighbors=2)
#         full_cx_data, full_cy_data = over_sampler.fit_resample(full_cx_data, full_cy_data)
#         full_cy_data = to_categorical(full_cy_data)
              
        
#         classifier = define_attack_model(2)
#         history = classifier.fit(full_cx_data, full_cy_data, epochs=EPS, batch_size=32, verbose=0)
        
        d=1
        pix = full_cx_data.shape[1]
        classifier, _ = attack_mlp(pix,d)
        history = classifier.fit(full_cx_data, full_cy_data, epochs=EPS, batch_size=32, verbose=0)
        

        #get predictions on real train and test data
        c_indices = np.where(checkmem_class_status==c)
        pred_y = classifier.predict(checkmem_prediction_vals[c_indices])
        c_pred_y = np.argmax(pred_y, axis=1)
        c_target_y = checkmem_membership_status[c_indices]
        
       
        target_membership.append(c_target_y)
        predicted_membership.append(c_pred_y)

    target_membership=np.array([item for sublist in target_membership for item in sublist])
    predicted_membership=np.array([item for sublist in predicted_membership for item in sublist])


    tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (target_membership,predicted_membership)   
    return tp, fp, fn, tn, precision, advj, acc, recall

In [106]:
def shokri_attack(attack_df, mem_validation, nmem_validation):
    
    predicted_membership, predicted_nmembership, true_membership, TP_idx, TN_idx  = [], [], [], [], []

    class_val = np.unique(attack_df['y'])
    ncval=attack_df.shape[1]-1
    
    for c_val in class_val:

        print(c_val)
        
        filter_rec_all = attack_df[(attack_df['y'] == c_val)]
        filter_rec_idx = np.array(filter_rec_all.index)
        
        attack_feat = filter_rec_all.iloc[:, 0:ncval]
        attack_class = filter_rec_all['membership']
             
        d=1
        pix = attack_feat.shape[1]
        
        attack_model, _ = attack_mlp(pix,d)
        
       
        history = attack_model.fit(attack_feat, attack_class, epochs=EPS, batch_size=32, verbose=0)
        
        mcval=mem_validation.shape[1]-1
        
        
        check_mem_feat = mem_validation[mem_validation['y']==c_val]
        check_nmem_feat = nmem_validation[nmem_validation['y']==c_val]
        
        if (len(check_mem_feat)!=0) and (len(check_nmem_feat)!=0):
        
            check_mem_feat_idx =  np.array(check_mem_feat.index)


            check_nmem_feat_idx =  np.array(check_nmem_feat.index)

            #print(check_nmem_feat_idx)
            #print(np.argmax(mpred,axis=1)==0)


            mpred = attack_model.predict(np.array(check_mem_feat))    
            predicted_membership.append(np.argmax(mpred,axis=1) )

            nmpred = attack_model.predict(np.array(check_nmem_feat))    
            predicted_nmembership.append(np.argmax(nmpred,axis=1) )        



            TP_idx.append(check_mem_feat_idx[np.where(np.argmax(mpred,axis=1)==1)[0]])

            TN_idx.append(check_nmem_feat_idx[np.where(np.argmax(nmpred,axis=1)==0)[0]])

    pred_members = np.array([item for sublist in predicted_membership for item in sublist])
    pred_nonmembers = np.array([item for sublist in predicted_nmembership for item in sublist])
    
    TP_idx_list = np.array([item for sublist in TP_idx for item in sublist])
    TN_idx_list = np.array([item for sublist in TN_idx for item in sublist])
    
    members=np.array(list(pred_members))
    nonmembers=np.array(list(pred_nonmembers))
    
    pred_membership = np.concatenate([members,nonmembers])
    ori_membership = np.concatenate([np.ones(len(members)), np.zeros(len(nonmembers))])
    
    return pred_membership, ori_membership, TP_idx_list, TN_idx_list

In [110]:
# trian target model
per_class_sample=150
channel=0   
EPS=50
act_layer=6
n_class = 100
VERBOSE = 0
test_ratio = 0.3

(target_trainX, target_trainY), (target_testX, target_testY), dim = load_target_data(target_dataset, test_ratio)
target_model,_ = build_purchase_dnn(n_class,dim)
#get_trained_keras_models(model, (target_trainX, target_trainY), (target_testX, target_testY), num_models=1)
history = target_model.fit(target_trainX, target_trainY, epochs=EPS, batch_size=32, verbose=VERBOSE)
score = target_model.evaluate(target_testX, target_testY, verbose=VERBOSE)
print('\n', 'Model test accuracy:', score[1])


 Model test accuracy: 0.7296666502952576


In [120]:
# prepare attack test data

members = []
nonmembers = []

memberX, memberY, nonmemberX, nonmemberY = load_attack_test_data(attack_test_members, attack_test_nonmembers)

# member
target_model_member_pred = target_model.predict(memberX, batch_size=32)
target_model_member_class = np.argmax(memberY, axis=1)
target_model_member_pred = np.vstack(target_model_member_pred)
#target_model_member_class = [item for sublist in target_model_member_class for item in sublist]
members.append(np.ones(len(target_model_member_pred)))
members = [item for sublist in members for item in sublist]


# nonmember
target_model_nonmember_pred = target_model.predict(nonmemberX, batch_size=32)
target_model_nonmember_class = np.argmax(nonmemberY, axis=1)
target_model_nonmember_pred = np.vstack(target_model_nonmember_pred)
#target_model_nonmember_class = [item for sublist in target_model_nonmember_class for item in sublist]
nonmembers.append(np.zeros(len(target_model_nonmember_pred)))
nonmembers = [item for sublist in nonmembers for item in sublist]

full_attack_test_pred_val = (target_model_member_pred, target_model_nonmember_pred)
full_attack_test_mem_status = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
full_attack_test_class_status = (np.array(target_model_member_class),np.array(target_model_nonmember_class))

print('\n pred', full_attack_test_pred_val)
print('\n class', full_attack_test_class_status)
print('\n mem status', full_attack_test_mem_status)

attack_test_data = (full_attack_test_pred_val, full_attack_test_mem_status,full_attack_test_class_status)

313/313 [==============================] - 1s 4ms/step

 pred (array([[2.98413169e-11, 3.17552042e-14, 2.61176128e-12, ...,
        1.19767781e-12, 5.54956092e-13, 2.99500080e-05],
       [1.54623638e-08, 1.35977216e-10, 1.17508484e-04, ...,
        2.67538422e-12, 1.07921831e-12, 8.04744726e-14],
       [3.56247523e-14, 2.98693125e-15, 9.08623360e-16, ...,
        9.36399985e-11, 9.79389920e-12, 1.40032539e-12],
       ...,
       [1.11653964e-09, 2.61938756e-14, 1.55246312e-06, ...,
        5.48449036e-13, 5.51795378e-16, 1.55160027e-08],
       [1.43374635e-11, 4.26041608e-13, 2.16723991e-12, ...,
        2.17559894e-11, 4.04710727e-12, 9.94139233e-12],
       [8.68660806e-08, 5.35840563e-08, 2.98836268e-03, ...,
        6.18522432e-13, 2.02693261e-11, 3.59012441e-14]], dtype=float32), array([[5.25158042e-12, 1.55048997e-06, 3.08961767e-09, ...,
        1.01382125e-09, 5.55571700e-10, 6.76230209e-12],
       [1.04933728e-10, 6.67927280e-15, 3.17102056e-09, ...,
        4.20191659e-1

In [278]:
# save attack test dataset to csv
df = pd.DataFrame()
mem = pd.Series(full_attack_test_pred_val[0][:,0])
nonmem = pd.Series(full_attack_test_pred_val[1][:,0])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
len(total)
df['prob_class_0'] = total
df = df.reset_index()
mem = pd.Series(full_attack_test_pred_val[0][:,1])
nonmem = pd.Series(full_attack_test_pred_val[1][:,1])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
df['prob_class 1'] = total

mem = pd.Series(full_attack_test_mem_status[0][:])
nonmem = pd.Series(full_attack_test_mem_status[1][:])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
df['mem_status'] = total

mem = pd.Series(full_attack_test_class_status[0][:])
nonmem = pd.Series(full_attack_test_class_status[1][:])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
df['class_status'] = total

df.drop("index", axis=1, inplace=True)


df.to_csv('attack_test_data.csv')

In [112]:
#prepare shadow dataset
n_shadow_models = 20
shadow_data_size = 10000
test_ratio = 0.3

load_shadow_data(shadow_dataset, n_shadow_models, shadow_data_size, test_ratio)

shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_tes

In [113]:
#train shadow model
per_class_sample=150
channel=0   
EPS=50
act_layer=6
n_class = 100
VERBOSE = 0
test_ratio = 0.3

n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init = train_shadow_models(n_shadow_models, n_class, dim, channel)

Shadow Model  0
Training shadow model 0
Shadow Train acc :  100.0 Shadow Test acc :  71.29999995231628
94/94 [==============================] - 0s 3ms/step
Shadow Model  1
Training shadow model 1
Shadow Train acc :  100.0 Shadow Test acc :  74.40000176429749
94/94 [==============================] - 0s 3ms/step
Shadow Model  2
Training shadow model 2
Shadow Train acc :  99.97143149375916 Shadow Test acc :  72.2000002861023
94/94 [==============================] - 0s 4ms/step
Shadow Model  3
Training shadow model 3
Shadow Train acc :  97.98571467399597 Shadow Test acc :  66.26666784286499
94/94 [==============================] - 0s 4ms/step
Shadow Model  4
Training shadow model 4
Shadow Train acc :  100.0 Shadow Test acc :  73.1333315372467
94/94 [==============================] - 0s 4ms/step
Shadow Model  5
Training shadow model 5
Shadow Train acc :  80.45714497566223 Shadow Test acc :  55.50000071525574
94/94 [==============================] - 0s 4ms/step
Shadow Model  6
Training shado

In [114]:
attack_df = prep_attack_data(n_attack_data)
attack_df

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,y,membership
0,6.541544e-09,1.174988e-04,4.346246e-08,3.301803e-04,3.626944e-09,2.409745e-07,1.580226e-07,8.283080e-06,2.442902e-06,4.205181e-08,...,1.054758e-09,6.547305e-07,9.407898e-04,9.973258e-01,3.270771e-07,1.051507e-04,1.226892e-05,6.049490e-07,95,1
1,5.547408e-10,4.827722e-10,4.904956e-09,1.147579e-10,8.457438e-12,6.471433e-12,5.096669e-10,1.361659e-10,9.337573e-10,9.030466e-08,...,8.905618e-08,1.324696e-10,3.149643e-09,8.320056e-10,3.835443e-07,4.170861e-07,1.625577e-05,1.751215e-06,85,1
2,1.733920e-09,2.809540e-08,9.842902e-10,7.071368e-11,9.491693e-11,3.591490e-12,1.035509e-09,1.391404e-08,7.793460e-11,6.907168e-10,...,1.955270e-10,6.344954e-10,1.612994e-10,6.682086e-08,1.080540e-10,7.624012e-07,2.918887e-04,2.421112e-11,60,1
3,5.699854e-07,5.283324e-11,1.340632e-07,1.915408e-10,1.822124e-04,6.592505e-12,3.998793e-08,1.597922e-09,1.688016e-09,8.662173e-10,...,1.062382e-11,4.718090e-11,3.240416e-12,2.931615e-10,1.450711e-12,1.605902e-10,3.955463e-12,3.835939e-12,20,1
4,2.272562e-08,1.262606e-12,2.823182e-09,1.278845e-11,7.455685e-11,1.315029e-13,9.146635e-10,3.338455e-15,9.232563e-12,2.473779e-08,...,3.266597e-08,2.475584e-10,2.046895e-12,5.940438e-15,3.333726e-08,1.257963e-10,1.152253e-11,2.189039e-09,48,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,4.942136e-08,5.101435e-09,3.858816e-09,9.396201e-11,2.810468e-10,4.035811e-14,1.103672e-08,2.481958e-13,1.289252e-09,1.988193e-07,...,8.084100e-08,2.099107e-09,4.235858e-09,9.735515e-14,1.215275e-06,6.657518e-11,1.044590e-09,2.800449e-06,48,0
199996,9.314193e-06,3.875714e-10,1.353274e-06,4.515317e-08,2.462291e-06,4.300380e-10,1.096163e-06,8.803803e-10,2.240690e-07,2.465790e-06,...,3.656181e-09,2.386638e-10,3.281247e-08,3.871282e-10,3.373403e-08,4.706728e-11,3.933095e-13,5.396639e-06,14,0
199997,1.097093e-08,8.175606e-09,3.953086e-10,1.845675e-09,7.099168e-11,2.389507e-12,1.184922e-08,5.211804e-11,1.348438e-09,7.405021e-08,...,3.253727e-06,3.544342e-06,6.817405e-07,1.336299e-10,1.131832e-04,2.960044e-08,8.514759e-06,2.727389e-06,79,0
199998,1.457679e-08,3.030750e-07,4.313308e-08,5.811122e-09,1.122036e-10,1.343821e-09,2.893942e-11,1.479642e-09,1.665378e-09,2.320726e-04,...,1.111406e-08,1.396644e-09,7.990046e-08,9.735260e-11,2.420566e-05,1.249579e-10,5.746050e-09,5.447636e-08,43,0


In [134]:
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
nmem_validation

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,y
0,5.251580e-12,1.550490e-06,3.089618e-09,2.265308e-08,3.170026e-08,1.031084e-02,4.649702e-10,9.896552e-01,1.313169e-09,4.429032e-10,...,1.004185e-16,3.399599e-12,1.564845e-11,2.459371e-09,1.639833e-06,3.243553e-13,1.013821e-09,5.555717e-10,6.762302e-12,7
1,1.049337e-10,6.679273e-15,3.171021e-09,6.189256e-14,4.556443e-07,6.804904e-12,3.055240e-09,2.365298e-12,3.223258e-11,5.143663e-11,...,5.241238e-09,4.407018e-10,3.363424e-10,5.079025e-12,7.379908e-12,1.064263e-10,4.201917e-11,5.147976e-13,3.644142e-11,52
2,6.728465e-11,3.142697e-13,3.153557e-14,1.791235e-13,1.008317e-12,3.015234e-15,1.291146e-09,3.509069e-13,1.904931e-14,1.118292e-16,...,2.609530e-09,2.808245e-08,4.857492e-11,2.417192e-14,3.680575e-09,1.260370e-14,1.033629e-05,6.086850e-08,5.664072e-09,87
3,6.500934e-09,4.689499e-09,3.941263e-03,6.813244e-09,3.803758e-03,4.176817e-09,7.135264e-04,2.966714e-06,3.088951e-04,1.749461e-07,...,1.370450e-07,6.212899e-12,3.594317e-12,2.134109e-07,4.222499e-07,6.900437e-09,1.821299e-07,7.015982e-06,4.398880e-08,24
4,5.112169e-12,1.624837e-17,4.551090e-09,1.882862e-16,1.254222e-13,5.018241e-14,8.860589e-12,8.627319e-15,4.184961e-11,1.877186e-11,...,9.764923e-08,1.521807e-12,4.531417e-17,1.124038e-16,5.622948e-15,3.318514e-13,3.633342e-12,2.549423e-15,3.960865e-12,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,3.781913e-10,6.541376e-11,5.897551e-10,1.679216e-10,1.178417e-05,1.305457e-09,2.700733e-07,2.045521e-08,2.512732e-10,4.840353e-13,...,1.038200e-10,7.603360e-10,1.126960e-07,1.228799e-08,1.344115e-07,3.768711e-11,1.835935e-06,4.451512e-07,1.260990e-08,54
9996,2.853237e-13,1.802704e-11,9.280368e-10,2.182457e-08,8.240011e-12,3.565267e-06,2.769353e-13,7.186834e-08,3.672361e-07,1.271133e-07,...,5.104006e-13,4.924649e-14,5.201058e-11,8.223761e-08,1.766298e-09,4.974094e-11,1.639730e-09,1.038645e-11,8.270240e-09,45
9997,4.148726e-10,9.963490e-01,8.602716e-10,9.938403e-04,1.822380e-09,5.324255e-05,8.899958e-09,2.590750e-03,5.592521e-08,3.592270e-08,...,3.304211e-13,1.840470e-10,2.606808e-08,1.315705e-06,4.582496e-06,2.061073e-09,9.693478e-10,2.566678e-07,3.724651e-10,1
9998,1.850620e-14,5.948074e-11,3.762144e-15,1.400853e-12,3.296175e-10,3.390383e-12,1.583681e-13,5.351064e-11,6.276414e-16,3.422593e-14,...,3.260824e-15,4.525114e-12,6.602087e-08,9.743842e-12,1.214545e-09,2.332410e-12,1.354049e-09,3.130175e-10,4.585324e-12,55


In [136]:
pred_membership, ori_membership, TP_idx_list, TN_idx_list = shokri_attack(attack_df, mem_validation, nmem_validation)

0
2/2 [==============================] - 0s 3ms/step
1
2/2 [==============================] - 0s 4ms/step
2
8/8 [==============================] - 0s 2ms/step
3
2/2 [==============================] - 0s 6ms/step
4
5/5 [==============================] - 0s 2ms/step
5
4/4 [==============================] - 0s 2ms/step
6
5/5 [==============================] - 0s 4ms/step
7
4/4 [==============================] - 0s 2ms/step
8
3/3 [==============================] - 0s 2ms/step
9
3/3 [==============================] - 0s 2ms/step
10
5/5 [==============================] - 0s 3ms/step
11
3/3 [==============================] - 0s 4ms/step
12
3/3 [==============================] - 0s 2ms/step
13
4/4 [==============================] - 0s 2ms/step
14
2/2 [==============================] - 0s 3ms/step
15
8/8 [==============================] - 0s 2ms/step
16
5/5 [==============================] - 0s 2ms/step
17
4/4 [==============================] - 0s 2ms/step
18
9/9 [==============================

2/2 [==============================] - 0s 3ms/step
79
2/2 [==============================] - 0s 4ms/step
80
4/4 [==============================] - 0s 2ms/step
81
3/3 [==============================] - 0s 2ms/step
82
4/4 [==============================] - 0s 2ms/step
83
2/2 [==============================] - 0s 3ms/step
84
3/3 [==============================] - 0s 2ms/step
85
3/3 [==============================] - 0s 2ms/step
86
3/3 [==============================] - 0s 3ms/step
87
4/4 [==============================] - 0s 2ms/step
88
2/2 [==============================] - 0s 3ms/step
89
3/3 [==============================] - 0s 3ms/step
90
4/4 [==============================] - 0s 2ms/step
91
2/2 [==============================] - 0s 3ms/step
92
3/3 [==============================] - 0s 2ms/step
93
2/2 [==============================] - 0s 3ms/step
94
2/2 [==============================] - 0s 3ms/step
95
2/2 [==============================] - 0s 3ms/step
96
2/2 [=======================

In [137]:
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)  

TP: 9385     FP: 6653     FN: 615     TN: 3347
PPV: 0.5852
Advantage: 0.2732


In [282]:
#save the prepared attack data on disk
np.savez(DATA_PATH + 'attack_purchase_data.npz', n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init)

/home/msnkhan/anaconda3/envs/r4-base/lib/python3.7/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


INFO:tensorflow:Assets written to: ram://a9382bc7-fd48-42a6-8436-af2bd2196281/assets


In [ ]:
# read stored attack model training data 
data_name = 'attack_adult_data.npz'
with np.load(DATA_PATH + data_name, allow_pickle=True) as f:
        n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init = [f['arr_%d' % i] for i in range(len(f.files))]

In [140]:
tp, fp, fn, tn, precision, advj, acc, recall = train_attack_model(n_attack_data, attack_test_data)
#target_membership, predicted_membership = train_attack_model(n_attack_data, attack_test_data)

Class :  0
5/5 [==============================] - 0s 2ms/step
Class :  1
4/4 [==============================] - 0s 2ms/step
Class :  2
16/16 [==============================] - 0s 1ms/step
Class :  3
4/4 [==============================] - 0s 2ms/step
Class :  4
10/10 [==============================] - 0s 1ms/step
Class :  5
6/6 [==============================] - 0s 2ms/step
Class :  6
8/8 [==============================] - 0s 1ms/step
Class :  7
7/7 [==============================] - 0s 2ms/step
Class :  8
6/6 [==============================] - 0s 2ms/step
Class :  9
6/6 [==============================] - 0s 2ms/step
Class :  10
10/10 [==============================] - 0s 2ms/step
Class :  11
6/6 [==============================] - 0s 2ms/step
Class :  12
7/7 [==============================] - 0s 2ms/step
Class :  13
8/8 [==============================] - 0s 2ms/step
Class :  14
5/5 [==============================] - 0s 2ms/step
Class :  15
16/16 [==============================] - 0s 1ms

In [284]:
precision

0.5193872315473136

In [ ]:
target_membership

In [ ]:
predicted_membership

In [ ]:
df = pd.DataFrame()
df['target'] = pd.Series(target_membership)
df['predicted'] = pd.Series(predicted_membership)

In [ ]:
df

In [ ]:
df.to_csv('test.csv')

In [ ]:
attack_data = n_attack_data
check_membership = attack_test_data
n_hidden=50
learning_rate=0.01
batch_size=200
epochs=50
model='nn'
l2_ratio=1e-7

In [ ]:
x, y,  classes = attack_data

train_x = x[0]
train_y = y[0]
test_x = x[1]
test_y = y[1]
train_classes = classes[0]
test_classes = classes[1]

#print(tra)


In [ ]:
checkmem_prediction_vals, checkmem_membership_status, checkmem_class_status = check_membership

checkmem_prediction_vals=np.vstack(checkmem_prediction_vals)
checkmem_membership_status=np.array([item for sublist in checkmem_membership_status for item in sublist])
checkmem_class_status=np.array([item for sublist in checkmem_class_status for item in sublist])

train_indices = np.arange(len(train_x))
test_indices = np.arange(len(test_x))
unique_classes = np.unique(train_classes)


predicted_membership, target_membership = [], []

In [ ]:
for c in unique_classes:
    print("Class : ", c)
    c_train_indices = train_indices[train_classes == c]
    c_train_x, c_train_y = train_x[c_train_indices], train_y[c_train_indices]
    c_test_indices = test_indices[test_classes == c]
    c_test_x, c_test_y = test_x[c_test_indices], test_y[c_test_indices]
    c_dataset = (c_train_x, c_train_y, c_test_x, c_test_y)  

    full_cx_data=(c_train_x,c_test_x)
    full_cx_data = np.vstack(full_cx_data)

    full_cy_data=(c_train_y,c_test_y)
    full_cy_data = np.array([item for sublist in full_cy_data for item in sublist])
    full_cy_data = to_categorical(full_cy_data)

    classifier = define_attack_model(2)
    history = classifier.fit(full_cx_data, full_cy_data, epochs=200, batch_size=32, verbose=0)
    #classifier.save('model/attack_model_class{}'.format(c))

    #get predictions on real train and test data
    c_indices = np.where(checkmem_class_status==c)
    predict_y = classifier.predict(checkmem_prediction_vals[c_indices])
    print(predict_y)
    c_pred_y = np.argmax(classifier.predict(checkmem_prediction_vals[c_indices]),axis=1)
    #c_pred_y = np.where(predict_y > 0.5, 1,0)
    #c_pred_y = classifier.predict_classes(checkmem_prediction_vals[c_indices])

    c_target_y = checkmem_membership_status[c_indices]


    target_membership.append(c_target_y)
    predicted_membership.append(c_pred_y)




In [ ]:
c_indices = np.where(checkmem_class_status==c)
predict_y = classifier.predict(checkmem_prediction_vals[c_indices], batch_size=32)
c_pred_y = np.where(predict_y > 0.5, 1,0)
c_target_y = checkmem_membership_status[c_indices]
target_membership.append(c_target_y)
predicted_membership.append(c_pred_y)

In [ ]:
target_membership=np.array([item for sublist in target_membership for item in sublist])
predicted_membership=np.array([item for sublist in predicted_membership for item in sublist])

In [ ]:
prety_print_result (target_membership,predicted_membership)

In [ ]:
np.vstack(predicted_membership).shape

In [ ]:
df = pd.DataFrame()
df['target'] = pd.Series(target_membership)
df['predicted'] = pd.Series(predicted_membership.reshape((len(predicted_membership))))

In [ ]:
df

In [ ]:
predicted_membership